In [86]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import re 

In [88]:
df = pd.read_csv("text.csv")

In [89]:
if df.columns[0] == "Unnamed: 0":
    df.rename(columns={"Unnamed: 0": "index"}, inplace=True)
df.set_index("index", inplace= True)

In [90]:

df["text"] = df["text"].str.lower()
df["text"] = df["text"].str.strip()
df['text'] = df['text'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))


In [91]:
stop_words = {
    'a', 'an', 'the', 'and', 'or', 'but', 'if', 'because', 'as',
    'what', 'which', 'this', 'that', 'these', 'those',
    'then', 'just', 'so', 'than', 'such', 'both', 'through', 'about',
    'for', 'is', 'of', 'while', 'during', 'to', 'from', 'in', 'on',
    'with', 'at', 'by', 'into', 'over', 'under', 'above', 'below',
    'up', 'down', 'out', 'off', 'again', 'further', 'once',
    'here', 'there', 'when', 'where', 'why', 'how',
    'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other',
    'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
    'than', 'too', 'very', 'can', 'will', 'just', 'don', 'should',
    'now', 'am', 'are', 'was', 'were', 'be', 'been', 'being', 'have',
    'has', 'had', 'do', 'does', 'did', 'having', 'doing', 'would',
    'could', 'should', 'might', 'must', 'shall', 'may', 'also', 'i'
    }

def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    text = ' '.join(tokens)
    return text
df['text'] = df['text'].apply(normalize_text)


In [102]:
df.head()

,text,label
index,,
0,feel really helpless heavy hearted,4
1,ive enjoyed able slouch relax unwind frankly n...,0
2,gave my internship dmrg feeling distraught,4
3,dont know feel lost,0
4,kindergarten teacher thoroughly weary my job a...,4
